In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
mnet = MobileNetV2(input_shape=(224, 224, 3), include_top=False, alpha=1)  # para controlar o tamanho da rede, pode mudar o parâmetro alpha
mnet

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

dataset, metadata = tfds.load('beans', as_supervised=True, with_info=True)
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

class_names = metadata.features['label'].names
print("Class names: {}".format(class_names))

In [ ]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

def resize(images, labels):
    images = tf.image.resize(images, size=(224, 224))
    labels = tf.one_hot(labels, depth=len(class_names))

    return images, labels

def transform_images(images, labels):
    images, labels = resize(images, labels)
    images, labels = normalize(images, labels)

    return images, labels

train_dataset =  train_dataset.map(transform_images)
validation_dataset =  validation_dataset.map(transform_images)
test_dataset =  test_dataset.map(transform_images)

In [ ]:
for image, labels in train_dataset.take(1):
    break
pred = mnet(tf.expand_dims(image, 0))
pred.shape

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

mnet.trainable = False
gap = GlobalAveragePooling2D()
dropout = Dropout(rate=0.25)
fc1 = Dense(units=len(class_names), activation=tf.nn.softmax)
layers =[
    mnet,  # feature extractor
    gap,  # no lugar do flatten, utilizamos GAP
    dropout,
    fc1,
]
model = tf.keras.models.Sequential(layers)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import numpy as np

BATCH_SIZE = 32

num_train_examples = metadata.splits['train'].num_examples
num_validation_examples = metadata.splits['validation'].num_examples
num_test_examples = metadata.splits['test'].num_examples

train_dataset = train_dataset \
    .cache() \
    .repeat() \
    .shuffle(num_train_examples) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)

validation_dataset = validation_dataset \
    .cache() \
    .repeat() \
    .shuffle(num_validation_examples) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
history = model.fit(
    train_dataset, epochs=10,
    steps_per_epoch=np.ceil(num_train_examples / BATCH_SIZE),  # esses passos só servem porque fizemos o .repeat() no dataset
    validation_data=validation_dataset,
    validation_steps=np.ceil(num_validation_examples / BATCH_SIZE),  # esses passos só servem porque fizemos o .repeat() no dataset
    workers=-1,
)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(8, 6))
ax[0].plot(history.history['loss'], label='loss')
ax[0].plot(history.history['val_loss'], label='val loss')
ax[0].legend(frameon=False)
ax[1].plot(history.history['accuracy'], label='acc')
ax[1].plot(history.history['val_accuracy'], label='val acc')
ax[1].legend(frameon=False)
fig.suptitle("Training loss and acc", fontweight='bold', fontsize=14);

In [ ]:
test_dataset = test_dataset \
    .cache() \
    .batch(BATCH_SIZE)
model.evaluate(test_dataset)